In [16]:
# 3, AR 1 simulation
import numpy as np

def ar1_simulate(rho, errors, y0):

    y = []
    y.append(y0)
    t = len(errors)
    for i in range(1,t):
        y.append(y[-1] * rho + errors[i])
    
    y = np.asarray(y)
    print(type(y))
    return y



In [36]:
errors = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
y0 = 1
errors

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

In [17]:


ar1_simulate(0.9,errors,y0)

<class 'numpy.ndarray'>


array([1. , 7.9])

In [130]:
# 2 Subsampled RV
import pandas as pd
def subsampled_rv(rets,k):
    n = len(rets)

    # k is the block size:
    r2 = []
    for i in range(k):
        r2.append((rets.iloc[i::k] ** 2).sum())
    
    r2 = pd.Series(r2).sum()
    num = n / k
    denom = n - k + 1
    multiplier = num / denom

    rv = multiplier * r2
    rv = float(rv)

    return rv






In [51]:

prices = pd.read_csv("mfe-formal-work-1-2020-2021.csv.gz", index_col="time",
compression="infer") 
prices.columns = pd.to_datetime(prices.columns)
ln_p = np.log(prices)
returns = (prices.diff()/prices).dropna()

In [131]:
subsampled_rv( returns["2012-01-03"]*100,1)

(5.024909504074747, 1.0, 23400.0, 23400)

In [113]:
len(returns["2011-01-03"]*100) - 1 - 1

23398

In [128]:
rv[1]["2012-01-03"]

5.024908264194439

In [107]:
m = 23400
rv = {}

samp = [23400, 11700,  7800,  5850,  4680,  3900,  2925,  2600,  2340,
        1950,  1800,  1560,  1300,  1170,   975,   936,   900,   780,
         650,   600,   585,   520,   468,   450,   390,   360,   325,
         312,   300,   260,   234,   225,   200,   195,   180,   156,
         150,   130,   120,   117,   104,   100,    90,    78,    75,
          72,    65,    60,    52,    50,    45,    40,    39,    36,
          30,    26,    25,    24,    20,    18,    15,    13]

for i in samp:
    step = m // i
    _rv = ((ln_p.iloc[::step].diff().dropna()*100)**2).sum()
    rv[step] = _rv
rv = pd.DataFrame(rv)